In [1]:
import sys
sys.path.append('../..')
from Library.lossSRM import LossAbs
import numpy as np
import scipy.stats as st
from scipy.optimize import minimize

In [2]:
class MCLossFunction(LossAbs):
    def __init__(self, X, alpha, c = None):
        self.__alpha = alpha
        super(MCLossFunction, self).__init__(X, c)    
    def shortfall_risk(self, m = None):
        m = self._check_argument(m)
        #This substract from the col i of X, m_i
        #X is a matrix of dim columns and N rows where N is the sample's length
        x_minus_m = np.subtract(self.X, m)
        #axis = 1 means we sum the elements of each row
        mean_sum = np.mean(np.sum(x_minus_m, axis = 1))
        pos_part = np.maximum(x_minus_m, 0)
        mean_sum_2 = np.mean(np.sum(pos_part ** 2, axis = 1))
        cross_term = 0.
        for i in range(self.dim):
            for j in range(i + 1, self.dim):
                cross_term += np.mean(np.multiply(pos_part[:, i], pos_part[:, j]))
        return mean_sum + 0.5 * mean_sum_2 + self.__alpha * cross_term
    
    def shortfall_risk_jac(self, m):
        m = self._check_argument(m)
        x_minus_m = np.subtract(self.X, m)
        pos_part = np.maximum(x_minus_m, 0) 
        pos_part_mean = np.mean(pos_part, axis = 0)
        cross_jac = []
        for i in range(self.dim):
            temp = 0.
            indic_i = np.sign(pos_part[:, i])
            for k in range(self.dim):
                if k != i:
                    temp += np.mean(np.multiply(pos_part[:, k], indic_i))
            cross_jac.append(temp)
        return pos_part_mean + 1. + cross_jac

In [5]:
#Case: rho = 0, alpha = 0. When alpha = 0 the results are independent of correlation
#We choose sigma_3 higher than sigma_1 = sigma_2. We expect m3 to be higher thatn m2=m1 as it is riskier.

#M is the length of the sample
M = 100000
rho = 0.
mu = [0., 0., 0.]
sigma = [[0.5, 0.5 * rho, 0.], [0.5 * rho, 0.5, 0.], [0., 0., 0.6]]

rv = st.multivariate_normal(mean = mu, cov = sigma, allow_singular = True)
distr = rv.rvs(size = M)

c = 1.
alpha = 0.
loss = MCLossFunction(distr, alpha, c)
maxiter = 3500

init =  np.zeros(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})
res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.45636848212879416
            Iterations: 10
            Function evaluations: 11
            Gradient evaluations: 10
     fun: -0.45636848212879416
     jac: array([1., 1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 11
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([-0.15318809, -0.15586116, -0.14731923])
